In [18]:
from langfuse import Langfuse
import json

# Khởi tạo client
langfuse = Langfuse(
    public_key="pk-lf-750d9727-b47b-4110-925d-9c00872618fe",
    secret_key="sk-lf-2c87067a-54eb-428d-b1e1-76db66870884",
    host="http://localhost:3000",
)

your_session_id = "1"

# Truy vấn traces và sắp xếp
traces_in_session = langfuse.api.trace.list(session_id=your_session_id, limit=100)
sorted_traces = sorted(traces_in_session.data, key=lambda x: x.timestamp)

chat_history = []

# Lặp qua từng trace để trích xuất
for trace in sorted_traces:
        ai_answer = ""
        user_question = ""

        # 1. Lấy câu trả lời của AI (logic này đã ổn)
        if isinstance(trace.output, str):
                ai_answer = trace.output
        elif isinstance(trace.output, dict):
                ai_answer = trace.output.get("content", "") or trace.output.get("response", "")

        # 2. Lấy câu hỏi của người dùng trực tiếp từ input dictionary
        # <--- THAY ĐỔI QUAN TRỌNG BẮT ĐẦU TỪ ĐÂY --->
        if isinstance(trace.input, dict):
            user_question = trace.input.get('question', '') # Truy cập trực tiếp vào key 'question'
        # <--- KẾT THÚC THAY ĐỔI QUAN TRỌNG --->

        # Thêm cặp hỏi-đáp vào lịch sử nếu có đủ thông tin
        if user_question and ai_answer:
                chat_history.extend([
                    {"role": "user", "content": user_question},
                    {"role": "assistant", "content": ai_answer},
                ])

        # Giới hạn số lượng cặp hỏi-đáp (ví dụ 6 cặp gần nhất)
        max_pairs = 6
        chat_history[-(max_pairs * 2):]



In [19]:
print(trace.output)

Hiện tại trong dữ liệu tôi đang có không liệt kê được các địa điểm thu gom pin cụ thể ở Đà Nẵng. Để biết thông tin chi tiết, bạn có thể:

- Liên hệ **Sở Bảo vệ môi trường – Đà Nẵng** hoặc **Công ty quản lý rác thải thành phố** để hỏi về các điểm thu gom pin đã được cấp phép.  
- Kiểm tra tại các **cửa hàng điện máy lớn** (ví dụ: Nguyễn Kim, Điện máy Xanh, MediaMart) – nhiều cửa hàng này có chương trình “đổi pin cũ lấy pin mới”.  
- Tra cứu trên website của **Bộ Tài nguyên và Môi trường** hoặc các **ứng dụng thu gom rác thải** (như VnPost, MyRecycle) để xem bản đồ các điểm thu gom gần bạn.  

**Mẹo an toàn khi xử lý pin (đặc biệt là pin lithium‑ion):**  
- Không để pin tiếp xúc trực tiếp với lửa hoặc nhiệt độ cao; nếu pin có dấu hiệu phồng, rò rỉ hoặc cháy, hãy đặt nó vào thùng kim loại kín và thông báo cho cơ quan thu gom.  
- Trước khi mang pin tới điểm thu gom, **bọc đầu cực bằng băng keo** để tránh ngắn mạch và nguy cơ phát nổ.  

Nếu bạn có thêm thông tin (ví dụ: tên dự án, tổ chức

In [20]:
print(trace.input)

{'question': 'Các địa điểm thu gom pin ở danang?', 'session_id': '1', 'user_id': '1'}


In [21]:
print(sorted_traces)

[TraceWithDetails(id='36bc874f46e3f9ce4d2c60eb7457e5f1', timestamp=datetime.datetime(2025, 10, 4, 17, 23, 33, 423000, tzinfo=datetime.timezone.utc), name='get_restapi_response', input={'question': 'What is llms', 'session_id': '1', 'user_id': '1'}, output='**LLMs (Large Language Models)**  \nLarge Language Models are advanced AI systems that learn patterns in text from massive datasets. They use neural network architectures (e.g., transformers) to predict the next word in a sentence, enabling tasks such as answering questions, writing essays, translating languages, and generating code. The “large” part refers to the huge number of parameters (often billions) and the extensive training data, which give these models a broad understanding of language and knowledge across many domains.', session_id='1', release=None, version=None, user_id='1', metadata={'resourceAttributes': {'telemetry.sdk.language': 'python', 'telemetry.sdk.name': 'opentelemetry', 'telemetry.sdk.version': '1.34.1', 'serv

In [22]:
chat_history

[{'role': 'user', 'content': 'What is llms'},
 {'role': 'assistant',
  'content': '**LLMs (Large Language Models)**  \nLarge Language Models are advanced AI systems that learn patterns in text from massive datasets. They use neural network architectures (e.g., transformers) to predict the next word in a sentence, enabling tasks such as answering questions, writing essays, translating languages, and generating code. The “large” part refers to the huge number of parameters (often billions) and the extensive training data, which give these models a broad understanding of language and knowledge across many domains.'},
 {'role': 'user', 'content': 'Bạn là ai'},
 {'role': 'assistant',
  'content': '**Bạn là ai?**  \nTôi là một trợ lý AI được lập trình chuyên sâu về khoa học môi trường, với trọng tâm vào các loại pin (điện lít, sạc lại, một lần sử dụng), quy trình thu gom, an toàn cháy nổ, công nghệ tái chế và trách nhiệm của nhà sản xuất (EPR). Tôi có thể cung cấp thông tin, tư vấn và phân t